In [9]:
import os
from glob import glob

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras import Model

## Hyperparameter Tunning

In [33]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Preprocess

In [34]:
import pandas as pd

In [35]:
train_df = pd.read_csv('train_dataset.csv')
train_df.head()

,name,class
0,../dataset/cifar/train\0_frog.png,frog
1,../dataset/cifar/train\10000_automobile.png,automobile
2,../dataset/cifar/train\10001_frog.png,frog
3,../dataset/cifar/train\10002_frog.png,frog
4,../dataset/cifar/train\10003_ship.png,ship


In [36]:
test_df = pd.read_csv('test_dataset.csv')
test_df.head()

,name,class
0,../dataset/cifar/test\0_cat.png,cat
1,../dataset/cifar/test\1000_dog.png,dog
2,../dataset/cifar/test\1001_airplane.png,airplane
3,../dataset/cifar/test\1002_ship.png,ship
4,../dataset/cifar/test\1003_deer.png,deer


In [37]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.3,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [38]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='name',
    y_col='class',
    target_size=input_shape[:2],
    batch_size=batch_size
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='name',
    y_col='class',
    target_size=input_shape[:2],
    batch_size=batch_size
)

Found 50000 validated image filenames belonging to 10 classes.
Found 10000 validated image filenames belonging to 10 classes.


In [39]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(input_shape=input_shape, filters=32, kernel_size=(3,3), strides=(1, 1), padding='SAME', activation='relu'),
    tf.keras.layers.MaxPool2D([2, 2], padding='same'),
    tf.keras.layers.Dropout(0.25),    
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), strides=(1, 1), padding='SAME', activation='relu'),
    tf.keras.layers.MaxPool2D([2, 2], padding='same'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

## Training

In [42]:
#step_per_epoch의 값을 넣는 방법
batch_size, len(train_df)//32 ,len(train_generator) 

(32, 1562, 1563)

In [40]:
model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator), # per epoch에 몇개의 batch를 돌거냐
        epochs=num_epochs,
        validation_data=test_generator,
        validation_steps=len(test_generator))

Epoch 1/10
1563/1563 [==============================] - 185s 118ms/step - loss: 1.6995 - accuracy: 0.3768 - val_loss: 1.4011 - val_accuracy: 0.4974
Epoch 2/10
1296/1563 [=======================>......] - ETA: 28s - loss: 1.4604 - accuracy: 0.4754- ETA: 32s -

KeyboardInterrupt: 